#### Introduction

Objective of lab:
> Create a dashboard application that contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart.

* **Task 1:** Add a Laucnh Site dropdown input component
* **Task 2:** Add a callback function to render <code>success-pie-chart</code> based on selected site dropdown
* **Task 3:** Add a Range slider to select Payload
* **Task 4:** Add a callback function to render the <code>success-payload-scatter-chart</code> scatter plot

#### Pre-requisites

In [2]:
# Run the below command in your terminal to install the Python packages required for running the application
# python3.11 -m pip install pandas dash

# Run the below command in your terminal to download the dataset (csv) file 
# wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

# Run the below command in your terminal to download a skeleton Dash app to be completed
# wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

# Running the above file will render a nearly blank page. 

Refer to the below lab if you want to refresh the basics about Plotly Dash components and callback functions.

<a href = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/Module%204/4.7_Dash_Interactivity.py">Plotly Dash Lab</a>

#### Building Application

In [40]:
# Importing required libraries

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [10]:
# Loading the dataset into a pandas dataframe

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
spacex_df = pd.read_csv(url, index_col = 0)
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [47]:
max_payload = ((spacex_df['Payload Mass (kg)'].max()//2500)+1)*2500
min_payload = int(spacex_df['Payload Mass (kg)'].min())
print(max_payload, min_payload)

10000.0 0


In [48]:
options = [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()]
options.append({'label': 'All Sites', 'value': 'All Sites'})

In [49]:
# Creating a Dash application

app = dash.Dash(__name__)

app.layout = html.Div(children = [html.H1('SpaceX Launch Records Dashboard',
                                          style = {'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
                                html.Br(),
                                
                                # Task 1: Add a dropdown list to enable launch site selection
                                html.Div(dcc.Dropdown(id = 'site-dropdown',
                                                      options = options,
                                                      placeholder = 'Select a site',
                                                      value = 'All Sites',
                                                      )),#style = {'width': '80%', 'padding': '3px', 'font-size': 25, 'textAlign': 'center'})),
                                html.Br(),

                                # Task 2: Add a pie chart to show total successfull launch count for all sites
                                # If a specific launch site is selected, then show the success vs failure count for the site
                                html.Div(
                                    dcc.Graph(id = 'success-pie-chart')),
                                html.Br(),

                                html.P('Payload Range (Kg):'),
                                # Task 3: Add a slider to select payload range
                                dcc.RangeSlider(id = 'payload-slider',
                                                min = min_payload, 
                                                max = max_payload, 
                                                step = 2500,
                                                value = [min_payload, max_payload],
                                                marks = {i: f'{i:.0f}' for i in range(min_payload, 10000 + 1, 2500)}),
                                html.Br(),

                                # Task 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id = 'success-payload-scatter-chart')),

])

# Task 2: Add a callback function for 'site-dropdown' as input, 'success-pie-chart' as output
@app.callback(Output(component_id = 'success-pie-chart', component_property = 'figure'),
               Input(component_id = 'site-dropdown', component_property = 'value'))

def generate_pie(site):
    if site == 'All Sites':
        success_pie_df = spacex_df.groupby('Launch Site', as_index = False)['class'].sum()
        fig = px.pie(success_pie_df, values = 'class', names = 'Launch Site')
        fig.update_layout(title = {'text': 'Total Success Launches by Site', 'x': 0.005, 'y': 0.98, 'xanchor': 'left', 'yanchor': 'top'})
        return fig
    else:
        pie_df = spacex_df[spacex_df['Launch Site'] == site]
        pie_df = pie_df.groupby('class', as_index = True)['class'].count()
        pie_df = pie_df.to_frame()
        pie_df.rename(columns = {'class': 'count'}, inplace = True)
        pie_df.reset_index(inplace = True)
        pie_df['class'].replace({0: 'Failure', 1: 'Success'}, inplace = True)
        fig = px.pie(pie_df, names = 'class', values = 'count')
        fig.update_layout(title = {'text': 'Success vs Failure for ' + site + 'Site', 'x': 0.005, 'y': 0.98, 'xanchor': 'left', 'yanchor': 'top'})
        return fig
    
# Task 4: Add a calback function for 'site-dropdown' and 'payload-slider' as inputs and 'success-payload-scatter-chart' as output
@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
               [Input(component_id = 'site-dropdown', component_property = 'value'),
                Input(component_id = 'payload-slider', component_property = 'value')])

def generate_scatter(site, range_):
    min_weight = range_[0]
    max_weight = range_[1]
    scatter_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_weight) & (spacex_df['Payload Mass (kg)'] <= max_weight)]
    if site == 'All Sites':
        fig = px.scatter(scatter_df, 
                    x = 'Payload Mass (kg)', 
                    y = 'class',
                    color = 'Booster Version Category' 
                    )
        fig.update_layout(title = {'text': 'Correlation between Payload and Success for All Sites', 'x': 0.005, 'y': 0.98, 'xanchor': 'left', 'yanchor': 'top'},)
        fig.update_xaxes(tickformat = 'd')
        return fig
    else:
        scatter_df = scatter_df[scatter_df['Launch Site'] == site]
        fig = px.scatter(scatter_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category')
        fig.update_layout(title = {'text': 'Correlation between Payload and Success for ' + site, 'x': 0.005, 'y': 0.98, 'xanchor': 'left', 'yanchor': 'top'})
        fig.update_xaxes(tickformat = 'd')
        return fig
    
if __name__ == '__main__':
    app.run_server()


/Users/anuj/Library/Python/3.9/lib/python/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

/Users/anuj/Library/Python/3.9/lib/python/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



#### Plotly Dash Reference

**Dropdown (Input) component**

Refer <a href = 'https://dash.plotly.com/dash-core-components/dropdown'>here</a> for more details about <code>dcc.Dropdown()</code>

**Range slider (Input) component**

Refer <a href = 'https://dash.plotly.com/dash-core-components/rangeslider'>here</a> for more details about <code>dcc.RangeSlider()</code>

**Pie chart (Output) component**

Refer <a href = 'https://plotly.com/python/pie-charts/'>here</a> for more details about plotly pie charts

**Scatter chart (Output) component**

Refer <a href = 'https://plotly.com/python/line-and-scatter/'>here</a> for more details about plotly scatter charts

#### End of File

In [3]:
# Questions:

# Q1 Which site has the largest successful launches?
# Answer - 

# Q2 Which site has the highest launch success rate?
# Answer - 

# Q3 Which payload range(s) has the highest launch success rate?
# Answer - 

# Q4 Which payload range(s) has the lowest launch success rate?
# Answer - 

# Q5 Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?
# Answer - 